# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [1]:
from chrisbase.io import BaseProjectEnv
from ratsnlp.nlpbook.classification.arguments import ClassificationDeployArguments

config = ClassificationDeployArguments(
    env=BaseProjectEnv(project_name="DeepKorean"),
    pretrained_model_path="model/pretrained/KcBERT-Base",
    downstream_model_home="model/finetuned/nsmc",
    downstream_model_file=None,
    max_seq_length=128,
).save_working_config()

In [2]:
from pathlib import Path

config = Path(config)
assert config.exists(), f"No config file: {config}"
args = ClassificationDeployArguments.from_json(config.read_text())
args.as_dataframe()

,ClassificationDeployArguments,value
0,env.env_type,BaseProjectEnv
1,env.hostname,dl012
2,env.hostaddr,129.254.182.78
3,env.python_path,/data/dlt/mambaforge/envs/DeepKorean-23.03/bin/python3.10
4,env.project_name,DeepKorean
5,env.working_path,/data/dlt/proj/DeepKorean-23.03
6,env.running_file,runs/1-doc_cls-infer.ipynb
7,pretrained_model_path,model/pretrained/KcBERT-Base
8,downstream_model_home,model/finetuned/nsmc
9,downstream_model_file,epoch=0-val_loss=0.246-val_acc=0.899.ckpt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [3]:
import torch
from transformers import BertConfig, BertForSequenceClassification

fine_tuned_model_ckpt = torch.load(
    Path(args.downstream_model_home) / args.downstream_model_file,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_path,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
model = BertForSequenceClassification(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_path,
    do_lower_case=False,
)
tokenizer

BertTokenizer(name_or_path='model/pretrained/KcBERT-Base', vocab_size=30000, model_max_length=300, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [5]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        positive_prob = round(prob[0][1].item(), 4)
        negative_prob = round(prob[0][0].item(), 4)
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)"
    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{negative_prob * 100}%",
    }

# 웹서비스 개시
아래처럼 실행해 인퍼런스 함수를 웹서비스로 만듭니다.

In [6]:
from ratsnlp.nlpbook.classification.deploy import get_web_service_app

service = get_web_service_app(inference_fn, ngrok_home=args.env.working_path)
service.run()

 * Serving Flask app 'ratsnlp.nlpbook.classification.deploy'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


/data/dlt/proj/DeepKorean-23.03/ngrok/ngrok3 http 5000
 * Running on https://5f5a-129-254-182-78.jp.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [22/Mar/2023 04:36:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Mar/2023 04:36:27] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Mar/2023 04:36:30] "POST /api HTTP/1.1" 200 -
